In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

def define_model(path_to_model):
    tokenizer = AutoTokenizer.from_pretrained(path_to_model)
    model = AutoModelForCausalLM.from_pretrained(path_to_model,
                                                 torch_dtype="auto",
                                                 device_map="auto")
    return tokenizer, model

/Users/mikhail/Desktop/Distillation/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Инициализация учителя

teacher_tokenizer, teacher_model = define_model('./teacher_model/')

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.69s/it]
The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


In [3]:
# Инициализация few-shot'a

few_shot_GSM8K = '''
Problem:
Olivia has $23. She bought five cupcakes for $3 each and a milkshake for $4. How much money does she have left?
Solution:
First, calculate the total cost of the cupcakes:
5 cupcakes × $3 = $15.
Add the cost of the milkshake: $15 + $4 = $19.
Subtract from her initial amount: $23 − $19 = $4.
So, Olivia has $4 left.
Final Answer: 4​

Problem:
A bakery sells cookies in packs of 6. If a customer buys 9 packs, how many cookies does the customer get in total?
Solution:
Each pack contains 6 cookies.
The customer buys 9 packs.
Total cookies = 6 × 9 = 54.
Final Answer: 54

Problem:
There are 42 students in a class. One-third of them are boys. How many girls are in the class?
Solution:
Number of boys = 42 ÷ 3 = 14.
Number of girls = total students − boys = 42 − 14 = 28.
Final Answer: 28

Problem:
A car travels 60 miles per hour. How many miles does it travel in 2 hours and 30 minutes?
Solution:
Convert 2 hours 30 minutes to hours: 2.5 hours.
Distance = speed × time = 60 × 2.5 = 150 miles.
Final Answer: 150

Problem:
James has 3 times as many marbles as Lisa. Together, they have 48 marbles. How many marbles does James have?

Solution:
Let Lisa have x marbles.
Then James has 3x marbles.
Together: x + 3x = 4x = 48.
So, x = 48 ÷ 4 = 12.
James has 3 × 12 = 36 marbles.
Final Answer: 36
'''

In [4]:
# Функция генерации ответа на запрос

def invoke_llm(type_of_model, task, type):
    if type_of_model == 'teacher':
        tokenizer = teacher_tokenizer
        model = teacher_model
    # Когда обучим ученика
    # elif type_of_model == 'student':
    #     tokenizer = supa_tokenizer
    #     model = supa_model
    elif type_of_model == 'student':
            tokenizer = student_tokenizer
            model = student_model
    else:
        return None
    
    if type == 'GSM8K':
        few_shot = few_shot_GSM8K
    # elif type == 'Coin Flip':
    #     few_shot = few_shot_Coin_Flip
    # elif type == 'CSQA':
    #     few_shot = few_shot_CSQA

    if type  == 'GSM8K':
        content = few_shot + '\n' + task
    elif type in ['CSQA', 'Coin Flip']:
        content = task
    
    messages = [{"role": "user", 
                "content": content}]
        
    text = tokenizer.apply_chat_template(messages,
                                         tokenize=False,
                                         add_generation_prompt=True,
                                         enable_thinking=False)
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    output_text = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
    return output_text

In [5]:
# Импорт данных

import polars as pl

# GSM8K
splits_GSM8K = {'train': 'main/train-00000-of-00001.parquet'}
df_train_GSM8K = pl.read_parquet("hf://datasets/openai/gsm8k/" + splits_GSM8K["train"])
questions_GSM8K = df_train_GSM8K['question'].to_list()
answers_GSM8K = df_train_GSM8K['answer'].to_list()

# CSQA
splits_CSQA = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df_train_CSQA = pl.read_parquet('hf://datasets/tau/commonsense_qa/' + splits_CSQA['train'])
questions_CSQA = df_train_CSQA['question'].to_list()
choices_CSQA = df_train_CSQA['choices'].to_list()
answers_CSQA = df_train_CSQA['answerKey'].to_list()

# Coin Flip
from datasets import load_dataset
dataset = load_dataset("skrishna/coin_flip")
df_train = dataset['train'].to_pandas()  # или .to_dict(), если удобнее
df_train_coinf_flip = pl.from_pandas(df_train)
question_coin_flip_test = df_train_coinf_flip['inputs'].to_list()
answer_coin_flip_test = df_train_coinf_flip['targets'].to_list()

In [6]:
# Предобработка coin flip (установка заглавных букв для имен)

def find_all_indices(text, substring):
    indices = []
    start_index = 0
    while True:
        index = text.find(substring, start_index)
        if index == -1:
            break
        indices.append(index+len(substring))
        start_index = index + 1
    return indices

for p in range(len(question_coin_flip_test)):
    question_coin_flip_test[p] = question_coin_flip_test[p].replace(' Q: ', '')
    question_coin_flip_test[p] = question_coin_flip_test[p].replace('  Is the', ' Is the') + 'Answer yes or no.'

    new_massive = []
    a_coin = 'A coin is heads up. '
    flip = 'flip the coin. '
    flips = 'flips the coin. '

    idx1 = find_all_indices(question_coin_flip_test[p], a_coin)[0]
    idx2 = find_all_indices(question_coin_flip_test[p], flip)
    idx3 = find_all_indices(question_coin_flip_test[p], flips)

    question_coin_flip_test[p] = question_coin_flip_test[p][:idx1] + question_coin_flip_test[p][idx1].upper() + question_coin_flip_test[p][idx1+1:]

    # Для flip the coin. 
    for fl in [idx2, idx3]:
        if isinstance(fl, list):
            if len(fl) != 0:
                for elem in fl:
                    question_coin_flip_test[p] = question_coin_flip_test[p][:elem] + question_coin_flip_test[p][elem].upper() + question_coin_flip_test[p][elem+1:]
        elif isinstance(fl, int):
            question_coin_flip_test[p] = question_coin_flip_test[p][:fl] + question_coin_flip_test[p][fl].upper() + question_coin_flip_test[p][fl+1:]

In [ ]:
# Генерация по таскам из GSM8K

import time
from tqdm import tqdm

with open(f'full_train_data_1.txt', 'a', encoding='utf-8', buffering=1) as f:
    for q, a in tqdm(zip(questions, answers), total=len(questions)):
        start_time = time.time()
        teacher_response = invoke_llm('teacher', q)
        f.write(teacher_response + '\n')
        f.write(f'-----'*20 + '\n')
        f.flush()
        end_time = time.time()
        elapsed_time = end_time - start_time

In [ ]:
# Генерация обученным учеником
from tqdm import tqdm

with open(f'train_data_CSQA_2.txt', 'w', encoding='utf-8', buffering=1) as f:
    for q, c, a in tqdm(zip(questions_CSQA[1700:7000], choices_CSQA[1700:7000], answers_CSQA[1700:7000]), total=len(questions_CSQA[1700:7000])):
        
        labels = c['label']
        text_labels = c['text']
        variants = ''

        for glyph, txt in zip(labels, text_labels):
            if glyph == labels[-1]:
                variants = variants + glyph + ' — ' + txt
            else:
                variants = variants + glyph + ' — ' + txt + ', '

        # print(f'{q + ' Here are the answer options: ' + variants}')

        teacher_response = invoke_llm('teacher', q + '\nHere are the answer options:\n' + variants, 'CSQA')
        correct_answer = a

        f.write(f'Correct answer from original table: {correct_answer}\n\n')
        f.write(f'<|im_start|>user\n{q + '\nHere are the answer options:\n' + variants}<|im_end|>\n')
        f.write(f'<|im_start|>assistant\n<think>\n\n</think>\n\n{teacher_response}<|im_end|>\n')
        f.write('-'*100 + '\n')
        f.flush()
        # print(f'Correct answer: {correct_answer}\n\nStudent solution: {teacher_response}')
        # print(f'-----'*20)

print("Готово!")

In [10]:
# Генерация по таскам Coin Flip
from tqdm import tqdm

# Генерация обученным учеником Coin Flip

with open(f'train_data_Coin_Flip2.txt', 'w', encoding='utf-8', buffering=1) as f:
    for q, a in tqdm(zip(question_coin_flip_test[6500:8000], answer_coin_flip_test[6500:8000]), total=len(question_coin_flip_test[6500:8000])):
        correct_answer = a
        teacher_response = invoke_llm('teacher', q, 'Coin Flip')
        if '**Answer: ' in teacher_response:
            f.write(f'Correct answer: {correct_answer}\n\n')
            f.write(f'<|im_start|>user\n{q}<|im_end|>\n')
            f.write(f'<|im_start|>assistant\n<think>\n\n</think>\n\n{teacher_response}<|im_end|>\n')
            f.write('-'*100 + '\n\n')
            f.flush()
            # print(f'Correct answer: {correct_answer}\n\nTeacher solution: {teacher_response}')
            # print(f'-----'*20)

print('Готово!')

100%|██████████| 1500/1500 [1:12:55<00:00,  2.92s/it]

Готово!
